## A Hands-on Workshop series in Machine Learning
### Session 2: More on `pandas`: Groupby operations
#### Instructor: Aashita Kesarwani

Please:
- Resize the Zoom and Web browser with Jupyter notebook so that you have both of them side by side. 
- Ask questions. Type in the chat window or raise your hand to speak.
- Answer questions and participate in the discussion - type in the chat window or unmute yourself to speak.
- Ask for help, whenever needed, for the exercises.
- Mute yourself when not speaking. 

Today's session:

1. More on `pandas`: Groupby operations 
2. One hot encoding for categorical features
3. An exercise on preprocessing the movie reviews from the [IMDb dataset](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) using regular expressions

##### Groupby operations
In the last session, we learned the basics of data manipulation using `pandas`. Today, we will learn the **split-apply-combine** operations by **grouping rows** of a dataframe

First we import the relevant python modules:

In [1]:
import numpy as np
import pandas as pd

# The following two modules matplotlib and seaborn are for plots
import matplotlib.pyplot as plt
import seaborn as sns # Comment this if seaborn is not installed
%matplotlib inline

# The module re is for regular expressions
import re

Loading the [Titanic dataset from Kaggle](https://www.kaggle.com/c/titanic) stored in the `csv` file as a dataframe using [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function.

In [2]:
path = 'data/'
df = pd.read_csv(path + 'titanic.csv')

We use `head()` function to peek into the first 5 rows (or any number of rows by using `head(n)`).

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


[Description for the columns](https://www.kaggle.com/c/titanic/data) is as follows.  

|Variable|	Definition|	Key|   
|:---  |:--- |:---|
|PassengerId| Passenger ID |
|Survived| 	Survival|	0 = No, 1 = Yes |
|Pclass	|Ticket class|	1 = 1st, 2 = 2nd, 3 = 3rd|
|Sex	|Sex|	
|Age	|Age in years	|
|SibSp	|# of siblings / spouses aboard the Titanic	|
|Parch	|# of parents / children aboard the Titanic	|
|Ticket	|Ticket number	|
|Fare	|Passenger fare	|
|Cabin	|Cabin number	|
|Embarked	|Port of Embarkation	|C = Cherbourg, Q = Queenstown, S = Southampton|

Use the functions [`isnull()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isnull.html) and [`sum()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html) on the dataframe to find out the number of missing values in each column.

In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Detecting missing values is an important step in Feature Engineering, that is preparing the features (independent variables) to use for building the machine learning models. The next step is to handle those missing values. 

1. If you consider filling the missing values, what are the possible options? 
2. Can you make use of other values in that column to fill the missing values? 
3. Can you make use of other values in that row as well as values in that column to fill the missing values?

As we saw earlier, the column *Age* has a lot of missing values. Those missing values can be filled with the median (or mean) age of all passengers.

* Can you think of other options to fill the missing age values for the passengers? 
* Does the title of the passengers' provide useful information about their age? 

In the exercise session in the last session, we created a new column *Title* that is derived from the column *Name* using regular expressions. Below is the compact repetition of the code to do the same.

In [6]:
df['Title'] = df['Name'].apply(lambda name: re.findall("\w+[.]", name)[0])

df.Title.replace({'Ms.': 'Miss.', 'Mlle.': 'Miss.', 'Dr.': 'Rare', 'Mme.': 'Mrs.', 
                  'Major.': 'Rare', 'Lady.': 'Rare', 'Sir.': 'Rare', 'Col.': 'Rare', 
                  'Capt.': 'Rare', 'Countess.': 'Rare', 'Jonkheer.': 'Rare', 
                  'Dona.': 'Rare', 'Don.': 'Rare', 'Rev.': 'Rare'}, inplace=True)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs.
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr.


Let us make use of the newly derived *Title* column to fill the age based on the title of the passengers. For this, it would be helpful to group the passengers based on their title to get the median age for each group and then fill the missing age values for passengers with the median age for their title.

### 1. [GroupBy object](https://pandas.pydata.org/pandas-docs/version/0.22/groupby.html)

We can use [`groupby()`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.groupby.html) to group the rows of the dataframe based on a column(s), say *Title*.

In [7]:
df.groupby('Title')

The resulting object is not a dataframe but **there are multiple ways to derive a dataframe from the grouped object**.

One of the ways to derive a dataframe from a groupby object is **by aggregation**, that is computing a summary statistic (or statistics) about each group. For example, we can get the median values for the columns in each group of titles.

In [8]:
df.groupby('Title').median()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Title,,,,,,,
Master.,345.0,1.0,3.0,3.5,1.0,1.0,29.0625
Miss.,388.0,1.0,3.0,21.0,0.0,0.0,15.7417
Mr.,466.0,0.0,3.0,30.0,0.0,0.0,9.3500
Mrs.,435.5,1.0,2.0,35.0,1.0,0.0,26.0000
Rare,627.0,0.0,1.0,48.5,0.0,0.0,27.7208


The median age vary greatly for each group ranging from 3.5 to 48 years.

In [17]:
df.groupby('Title').mean()

PassengerId  Survived        Age     SibSp     Parch  \
Title   Pclass                                                         
Master. 1        518.333333  1.000000   5.306667  0.666667  2.000000   
        2        463.555556  1.000000   2.258889  0.888889  1.222222   
        3        388.285714  0.392857   5.350833  2.928571  1.357143   
Miss.   1        436.812500  0.958333  29.744681  0.375000  0.562500   
        2        438.514286  0.942857  22.560606  0.257143  0.542857   
        3        391.166667  0.500000  16.123188  1.009804  0.529412   
Mr.     1        441.186916  0.345794  41.580460  0.280374  0.252336   
        2        445.901099  0.087912  32.768293  0.307692  0.131868   
        3        461.416928  0.112853  28.724891  0.285266  0.125392   
Mrs.    1        488.953488  0.976744  40.400000  0.767442  0.372093   
        2        447.024390  0.902439  33.682927  0.682927  0.658537   
        3        420.523810  0.500000  33.515152  0.619048  1.452381   
Rare    1        596.733333  0.533333  47.571429  0.466667  0.066667   
        2        453.875000  0.000000  42.000000  0.125000  0.125000   

                      Fare  MedianAge  
Title   Pclass                         
Master. 1       117.802767        3.5  
        2        27.306022        3.5  
        3        28.177232        3.5  
Miss.   1       121.694356       21.0  
        2        20.182500       21.0  
        3        15.248043       21.0  
Mr.     1        68.034385       30.0  
        2        19.252564       30.0  
        3        11.299762       30.0  
Mrs.    1        92.615505       35.0  
        2        23.496139       35.0  
        3        18.233529       35.0  
Rare    1        48.035280       48.5  
        2        16.796875       48.5

The most common way to derive a dataframe from a groupby object is by transformation using [`transform()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html).. This returns a dataframe with the same number of rows as the original dataframe. The `transform("median")` method operates on the numerical columns and replaces the original values with the median value for the corresponding group.

In [10]:
df.groupby('Title').transform("median")

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,466.0,0,3,30.0,0,0,9.3500
1,435.5,1,2,35.0,1,0,26.0000
2,388.0,1,3,21.0,0,0,15.7417
3,435.5,1,2,35.0,1,0,26.0000
4,466.0,0,3,30.0,0,0,9.3500
...,...,...,...,...,...,...,...
886,627.0,0,1,48.5,0,0,27.7208
887,388.0,1,3,21.0,0,0,15.7417
888,388.0,1,3,21.0,0,0,15.7417
889,466.0,0,3,30.0,0,0,9.3500


Our goal is to create a new column *MedianAge* which consists of the groupwise median age depending on the passengers' title. So, we use [`transform()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html) only on the *Age* column instead of the whole dataframe.

In [16]:
df.groupby('Title')['Age'].transform("median")

0      26.0
1      40.0
2      18.0
3      40.0
4      26.0
       ... 
886    46.5
887    30.0
888    18.0
889    40.0
890    26.0
Name: Age, Length: 891, dtype: float64

So far, our original dataframe has not really changed.

In [12]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.


We create a new column *MedianAge* using [`groupby()`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.groupby.html) and [`transform()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html) methods are demonstrated above.

In [13]:
df['MedianAge'] = df.groupby('Title')['Age'].transform("median")
df.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,MedianAge
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.,30.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.,35.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,21.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs.,35.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr.,30.0
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Mr.,30.0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mr.,30.0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Master.,3.5
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Mrs.,35.0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Mrs.,35.0


Now we fill in the missing values in the *Age* column using the values in the *MedianAge* column.

In [18]:
df['Age'] = df['Age'].fillna(df['MedianAge'])
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,MedianAge
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.,30.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.,35.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,21.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs.,35.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr.,30.0


We drop off the *MedianAge* column since we no longer need it and then check for the missing values.

In [19]:
df = df.drop('MedianAge', axis=1)
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Title            0
dtype: int64

There are no missing values in the Age column!

In the next section, you will use the `groupby` object and create a new feature.

### 2. Feature engineering on the Titanic dataset to create a new column for group size 

In this exercise, we are going to create a new column called `GroupSize` to get the size of the group for each passenger. We consider groups to be either family members or those traveling on the same ticket. This feature is derived using three columns *SibSp*, *Parch* and *Ticket*. 

[Notes](https://www.kaggle.com/c/titanic/data) for the two features *SibSp* and *Parch* are as follows.

> SibSp: The dataset defines family relations in this way...  
Sibling = brother, sister, stepbrother, stepsister  
Spouse = husband, wife (mistresses and fiancés were ignored)
> 
> Parch: The dataset defines family relations in this way...  
Parent = mother, father  
Child = daughter, son, stepdaughter, stepson  
Some children travelled only with a nanny, therefore parch=0 for them.

Create a new column named *Family* by adding the columns *SibSp* and *Parch* and then add 1 to it. Hint: Use `df["New_column"] = df["column_1"] + df["column_2"] + 1`.

Now we check the survival rates with respect to the family size.

In [ ]:
sns.barplot(x='Family', y='Survived', data=df);

Some passengers that appear to be traveling alone by account of their family size were part of a group traveling on the same ticket. To see this, get all the passengers traveling on the ticket "1601" (there are 7 of them).

One can check that there are many tickets shared among passengers that may or may not be family members.

In [ ]:
df['Ticket'].value_counts()[:15]

Create a new column named *TicketCount* that counts the total number of passengers traveling in each passengers' ticket.

Hint: 
- First group passengers based on their tickets using `groupby()` on the *Ticket* column.
- For the grouped object, pick any column that has no missing values.
- Use `transform()` for this unique identifier column with the function `"count"` to create a new column *TicketCount*.

For example, we created *MedianAge* using the following code:   
```df['MedianAge'] = df.groupby('Title')['Age'].transform("median")```

Let us peek into the dataframe.

In [ ]:
df.head()

Plot the survival rates based on the *TicketCount* using [`sns.barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) (We used this function in the last session. Please refer to the notebook from session 1 if needed).

It does seem that the number of co-travelers have an impact on the survival rates.

Create a new column named *GroupSize* by picking the maximum value among the columns *Family* and *TicketCount*.   
Note: We consider groups to be either family members or those traveling on the same ticket.   
Hint: Use built-in [`max()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.max.html) function for pandas on the two relevant columns with the appropriate value for the `axis` parameter. 

Plot the survival rates based on the GroupSize using [`sns.barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html).

Check the number of rows where Groupsize is not equal to Family. Similarly, check the number of rows where TicketCount is not equal to Family.

In [ ]:
df[df['GroupSize'] != df['Family']].shape[0], df[df['GroupSize'] != df['TicketCount']].shape[0]

The output must be `(97, 191)`. Check your above code, if you get a different output.

### 3. Encoding categorical variables

Let us check the datatype of each column. Hint: Use [`dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html).

In [20]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
Title           object
dtype: object

In machine learning, our models usually take numbers as inputs rather than strings. We have to convert categorical data into a form the model can recognize.

We convert the gender values to numerical values 0 and 1 using [`replace`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html) with a suitable dictionary. 

In [ ]:
df = df.replace({'male': 0, 'female': 1})
df.head()

What can go wrong with randomly assigning numbers to categories?

There are two kinds of categorical variables based on whether the categories possess an inherent order or not:
* Ordinal categorical variables
* Inordinal categorical variables

For example, passengers' ticket class `Pclass` takes the values 1, 2, and 3. These three categories have an inherent order and hence it is an ordinal categorical variable. On the other hand, gender takes two values - male and female, which have no intrinsic ordering and hence it is an inordinal categorical variable.

Does it mean that we can simply treat the ordinal variables such as `Pclass` as another numerical variable? Can you think of any problem this may cause in our model?

Other than a natural order, number also possess certain other properties. For example, the difference between the numbers 1 and 2 is the same as the difference between the numbers 2 and 3. 
$$ 2-1 == 3-2$$

Can we make the same claim for the categories labeled $1, 2,$ and $3$ in our ordinal variables `Pclass`?

So, converting categories to numbers means adding untrue assumptions that may or may not adversely affect our model. 

To address this, the commonly used method is one-hot encoding. In this method, we build a one-hot encoded vector with dimension equal to the number of classes in the categories. This vector consists of all 0's except for a 1 corresponding to the class of the instance. For example, the *Embarked* column will have one-hot encoded vectors of [1,0,0], [0,1,0] and [0,0,1] representing each of the three possible ports. 

How will this look in our dataset?  
Instead of a single column for the port of embarkment, we will have three columns corresponding to each port. The values in these columns will be $0$ or $1$. For each row, there will be only one $1$ among these three columns.

One-hot encoding is accomplished in pandas using [`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) as given below. It simply creates a column for each class of a categorical variable.

In [ ]:
pd.get_dummies(df['Embarked']).head()

We want the column names to be `'Port_C', 'Port_Q', 'Port_S'`. Make use of the [`prefix` ](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) keyword in `get_dummies` to alter the column names and save the one-hot encoded vectors to a new dataframe named `port_df`.

In [ ]:
df.head()

Notes:
- One of the columns in the one-hot encoding obtained in the above manner is always redundant. In case of features with just two classes such as gender in our dataset, one-hot encoding is not truly useful. One of its column is same as what we obtained by simply replacing classes with 0 and 1 and the other is redundant.  
- The main disadvantage of using one-hot encoding is the increase in the number of features that can negatively affect our model which we will discuss in the later sessions.

#### Acknowledgment:
* [Titanic dataset from Kaggle](https://www.kaggle.com/c/titanic) dataset openly available in Kaggle is used in the exercises.
